Going to perform a two-way repeated measures ANOVA (mixed). The between measures variables will be the drug they are taking (aspiring vs.  Rivaroxaban). The within measures variable will be the 3 times the patient is checked during follow-up appointments. 

I'll be assessing how the treatment (drug) type and time influence certain scores, such as LDH concentration, etc. 

Load libraries

In [1]:
library(rstatix)
library(ez)

print("complete")


Attaching package: ‘rstatix’

The following object is masked from ‘package:stats’:

    filter

Registered S3 methods overwritten by 'lme4':
  method                          from
  cooks.distance.influence.merMod car 
  influence.merMod                car 
  dfbeta.influence.merMod         car 
  dfbetas.influence.merMod        car 


[1] "complete"


Load dataset

In [2]:
ldh_original <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/LDH_2wayANOVA.csv')
ldh_cleaned <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/LDH_2wayANOVA_cleaned.csv')
ldh_cleaned_cleaned <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/LDH_2wayANOVA_cleaned_cleaned.csv')

dd_original <- read.csv(file = "/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/DDimer_2wayANOVA.csv")
crp_original <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/Crp_2wayANOVA.csv')
b2m_original <- read.csv(file = '/Users/anthonyquint/Desktop/LHSC_Work_Folder/Rhythm/TIme_varying_data_part2/b2m_2way_ANOVA.csv')


dataa <- ldh_cleaned_cleaned
select(dataa, treatment, visit_lab, score)

treatment,visit_lab,score
2,1,204
2,2,185
2,3,206
1,1,144
1,2,145
1,3,151
1,1,373
1,2,264
1,3,302
2,1,171


Convert everything other than score into factors

In [3]:
convert_as_factor(dataa$study_id)
convert_as_factor(dataa$treatment)
convert_as_factor(dataa$visit_lab)

[1] 3     3     3     5     5     5     8     8     8     10    10    10   
[13] 522-7 522-7 522-7 524-2 524-2 524-2 524-3 524-3 524-3 524-4 524-4 524-4
[25] 524-5 524-5 524-5 524-6 524-6 524-6
Levels: 10 3 5 522-7 524-2 524-3 524-4 524-5 524-6 8

[1] 2 2 2 1 1 1 1 1 1 2 2 2 2 2 2 1 1 1 1 1 1 2 2 2 1 1 1 2 2 2

[1] 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3

Summary Statistics

In [4]:
summary<-dataa %>%
group_by(treatment,visit_lab) %>%
get_summary_stats(score, type = "mean_sd")
data.frame(summary)

treatment,visit_lab,variable,n,mean,sd
1,1,score,5,199.8,104.248
1,2,score,5,188.6,60.657
1,3,score,5,189.0,74.273
2,1,score,5,190.0,27.249
2,2,score,5,204.0,29.138
2,3,score,5,202.0,23.906


Outlier detection

In [5]:
outlier<-dataa %>%
group_by(treatment,visit_lab) %>%
identify_outliers(score)
data.frame(outlier)
print("Here are the outliers. Decide what to do with them later. Most likely will need to take them out in the future")


treatment,visit_lab,study_id,score,is.outlier,is.extreme
1,1,8,373,TRUE,TRUE
1,3,8,302,TRUE,FALSE
2,3,10,163,TRUE,FALSE


[1] "Here are the outliers. Decide what to do with them later. Most likely will need to take them out in the future"


Normality Checking

In [6]:
normality<-dataa %>%
group_by(treatment,visit_lab) %>%
shapiro_test(score)
data.frame(normality)
print("need p > 0.05 to satisfy normality assumption")

treatment,visit_lab,variable,statistic,p
1,1,score,0.8738572,0.2823956
1,2,score,0.9633031,0.8307762
1,3,score,0.9551611,0.7739517
2,1,score,0.9278787,0.5819776
2,2,score,0.9738572,0.8993833
2,3,score,0.8909063,0.3616809


[1] "need p > 0.05 to satisfy normality assumption"


Applying anova test with sphericity corrections 

In [7]:
#res.aov <- anova_test(
#data = dataa, dv = score, wid = study_id,
#within = c(treatment, visit_lab)
#)
#get_anova_table(res.aov)

Applying ANOVA with another package

In [8]:
newModel<-ezANOVA(
    data = dataa, dv = .(score), wid = .(study_id), 
    within = .(visit_lab), between = .(treatment), detailed = FALSE, type = 2)
newModel

Warning message:
“"visit_lab" will be treated as numeric.”Warning message:
“"treatment" will be treated as numeric.”Warning message:
“There is at least one numeric within variable, therefore aov() will be used for computation and no assumption checks will be obtained.”

Effect,DFn,DFd,F,p,p<.05,ges
treatment,1,8,0.029037960,0.8689222,,3.437595e-03
visit_lab,1,8,0.003468709,0.9544797,,2.153618e-05
treatment:visit_lab,1,8,1.252204076,0.2956030,,7.714750e-03


Pairwise comparisons between treatment groups at each time point

In [9]:
dataa %>%
pairwise_t_test(
score ~ treatment, paired = TRUE,
p.adjust.method = "bonferroni"
)

.y.,group1,group2,n1,n2,statistic,df,p,p.adj,p.adj.signif
score,1,2,15,15,-0.2710301,14,0.79,0.79,ns


Comparisons for the time variable

In [10]:
dataa %>%
pairwise_t_test(
score ~ visit_lab, paired = TRUE,
p.adjust.method = "bonferroni"
)

.y.,group1,group2,n1,n2,statistic,df,p,p.adj,p.adj.signif
score,1,2,10,10,-0.09704040,9,0.925,1,ns
score,1,3,10,10,-0.05808749,9,0.955,1,ns
score,2,3,10,10,0.13848100,9,0.893,1,ns
